In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

from scipy.special import jv
from scipy.special import yv
from scipy.optimize import root

$$(i) \frac{\partial s}{\partial t} = k\frac{\partial^2 s}{\partial x^2} +\frac{Q}{A}\frac{\partial s}{\partial x}$$

$$(ii) \frac{\partial \zeta}{\partial t} = \kappa \frac{\partial^2 \zeta}{\partial r^2} + \frac{1}{r} \left (\kappa-\frac{Q}{\pi} \right )\frac{\partial \zeta}{\partial r}$$

In [3]:
Q = 150  # m/s^3
A = 2000 # m^2
k = 400
kappa = k
R = 10000
L = 10000

D = 1

pi = np.pi

a = A/(pi*D)

In [4]:
def ssx(x):
    pwr = Q/(kappa*pi*D)
    d1 = 1/(R**pwr -a**pwr*np.exp(-Q*L/(k*A)))
    c1 = d1*a**pwr
    c2 = -np.exp(-Q*L/(k*A))*c1
    return c1 * np.exp(-Q*x/(k*A)) + c2

def ssr(r):
    pwr = Q/(kappa*pi*D)
    d1 = 1/(R**pwr -a**pwr*np.exp(-Q*L/(k*A)))
    d2 = 1-d1*R**pwr
    return d1*r**pwr + d2

def ssx0(x):
    pwr = 2*Q/(kappa*pi*D)
    d1 = 1/(R**pwr -a**pwr*np.exp(-2*Q*L/(k*A)))
    c1 = d1*a**pwr
    c2 = -np.exp(-2*Q*L/(k*A))*c1
    return c1 * np.exp(-2*Q*x/(k*A)) + c2

def ssr0(r):
    pwr = 2*Q/(kappa*pi*D)
    d1 = 1/(R**pwr -a**pwr*np.exp(-2*Q*L/(k*A)))
    d2 = 1-d1*R**pwr
    return d1*r**pwr + d2

### Finding eigenfunctions and -values

In [5]:
order = Q/(2*kappa*pi)

def phi_r(r,par):
    return (r)**(order)*jv(order, np.sqrt(par[0]/kappa)*(r)) + par[1]*(r)**(order)*yv(order, np.sqrt(par[0]/kappa)*(r))

def phi_r_prime0(r,par):
    c = order
    w = np.sqrt(par[0]/kappa)
    d = par[1]
    
    T1 = d*w*r*yv(c-1, w*r)
    T2 = 2*c*d*yv(c, w*r)
    T3 = -d*w*r*yv(c+1,w*r)
    T4 = w*r*jv(c-1,w*r)
    T5 = 2*c*jv(c,w*r)
    T6 = -w*r*jv(c+1,w*r)
    return 0.5*r**(c-1)*(T1+T2+T3+T4+T5+T6)

def phi_r_dprime0(r,par):
    c = order
    w = np.sqrt(par[0]/kappa)
    d = par[1]
    
    T11 = (c**2+c-w**2*r**2)*yv(c, w*r)/(w**2*r**2)
    T12 = yv(c-1,w*r)/(w*r)
    T1 = d*w**2*r**c*(T11 - T12)
    
    T21 = (c**2+c-w**2*r**2)*jv(c, w*r)/(w**2*r**2)
    T22 = jv(c-1,w*r)/(w*r)
    T2 = w**2*r**c*(T21-T22)
    
    T3 = (c-1)*c*d*r**(c-2)*yv(c,w*r) + c*d*w*r**(c-1)*(yv(c-1,w*r) - yv(c+1,w*r))
    T4 = (c-1)*c*r**(c-2)*jv(c,w*r) + c*w*r**(c-1)*(jv(c-1,w*r) - jv(c+1,w*r))
    return (T1+T2+T3+T4)

In [6]:
good_roots_r = np.array([[3.94030436e-05,7.61848195e-01], #n=0
                          [1.71476791e-04,1.36154281e+00], #n=1
                          [3.94842613e-04,2.49685017e+00], #n=2
                          [7.08883157e-04,6.97900847e+00], #n=3
                          [1.11333787e-03,-1.12220582e+01], #n=4
                          [1.60807412e-03,-3.06309775e+00], #n=5
                          [2.19302e-03,-1.67100131e+00], #n=6
                          [2.86812e-03,-1.05017646e+00], #n=7
                          [3.63335e-03,-0.66912985e+00], #n=8
                          [4.4887e-03,-0.38862351e+00], #n=9
                          [5.43415e-03,-0.15366476e+00], #n=10
                          [6.46968e-03,0.065137e+00], #n=11
                          [7.5953e-03,0.2897497e+00], #n=12
                          [8.81099e-03,0.54445193e+00], #n=13
                          [1.011676e-02,0.86784664e+00], #n=14
                          [1.15126e-02,1.34271697e+00], #n=15
                          [1.29985e-02,2.21252377e+00], #n=16
                          [1.457447e-02,4.71344792e+00], #n=17
                          [1.62405098e-02,-2.61426615e+02], #n=18
                          [1.799661e-02,-4.44488786e+00]]) #n=20


In [7]:
N_r = 19
N_x = 19

#We now count from n=0 for simplicity and to add confusion :)

def phi_r_n(r,n):
    if n == 0:
        return ssr(r)
    return phi_r(r, good_roots_r[n-1])

def labda_r_n(n):
    if n == 0:
        return 0
    return good_roots_r[n-1][0]

def phi_r_prime(r,n):
    if n==0:
        return None
    return phi_r_prime0(r, good_roots_r[n-1])

def phi_r_dprime(r,n):
    if n==0:
        return None
    return phi_r_dprime0(r, good_roots_r[n-1])


def phi_x_n(x,n):
    if n==0:
        return ssx(x)
    return np.exp(-Q/(2*k*A)*x)*np.sin(n*pi*x/L)

def labda_x_n(n):
    if n == 0:
        return 0
    return (Q/A)**2/(4*k) + k*(n*pi/L)**2

def phi_x_prime(x,n):
    if n==0:
        return None
    return n*pi/L

def phi_x_dprime(x,n):
    if n==0: return 0
    return -2*n*pi/L*(-Q)/(2*k*A)


In [8]:
# amp0 = 2*Q*c1/(k*A)
f0 = ssx0(0) 


In [9]:
# def psi(r,t):
#     return g(t)*(r-R) + 1

# def xi(x,t):
#     return -g(t)*(x-L)

# def u(x):
#     return ssx0(x) - xi(x,0)

# def v(r):
#     return ssr0(r) - psi(r,0)

In [10]:
def inner_x(n,m):
    return integrate.quad(lambda x: phi_x_n(x,m)*phi_x_n(x,n), 0, L)[0]

def inner_r(n,m):
    return integrate.quad(lambda r: phi_r_n(r,m)*phi_r_n(r,n), a, R)[0]

G_x = np.array([[inner_x(n,m) for n in range(N_x)] for m in range(N_x)])
G_r = np.array([[inner_r(n,m) for n in range(N_r)] for m in range(N_r)])

labda_x = np.array([labda_x_n(n) for n in range(N_x)])
labda_r = np.array([labda_r_n(n) for n in range(N_r)])

inv_x = np.linalg.inv(G_x)
inv_r = np.linalg.inv(G_r)


In [11]:
a1 = -A*L/Q
a2 = a*(a-R)/(kappa-Q/(pi*D))

b0 = np.sum([inv_x[0,j] * integrate.quad(lambda x: (ssx0(x))*phi_x_n(x,j), 0, L)[0] for j in range(N_x)])
b1 = np.sum([inv_x[0,j] * integrate.quad(lambda x: -Q/A*(x-L)*phi_x_n(x,j), 0, L)[0] for j in range(N_x)])
b3 = np.sum([inv_x[0,j] * integrate.quad(lambda x: phi_x_n(x,j), 0, L)[0] for j in range(N_x)])

c0 = np.sum([inv_r[0,j] * integrate.quad(lambda r: (ssr0(r))*phi_r_n(r,j), a, R)[0] for j in range(N_r)])
c1 = np.sum([inv_r[0,j] * integrate.quad(lambda r: a/(kappa-Q/(pi*D))*(R-r)*phi_r_n(r,j), a, R)[0] for j in range(N_r)])
c3 = np.sum([inv_r[0,j] * integrate.quad(lambda r: a/r*phi_r_n(r,j), a, R)[0] for j in range(N_r)])

In [12]:
alpha = b3 + b1 - c3 - c1
beta = b1 - c1 + a1 - a2

print(alpha/beta)

0.04783860354548853


In [14]:
I0 = [sum([inv_x[n,j]*integrate.quad(lambda x: (ssx0(x)-f0*(1-x/L))*phi_x_n(x,j), 0, L)[0] for j in range(N_x)]) for n in range(N_x)]
I1 = [sum([inv_x[n,j]*integrate.quad(lambda x: (1-x/L)*phi_x_n(x,j), 0, L)[0] for j in range(N_x)]) for n in range(N_x)]
I2 = [sum([inv_x[n,j]*integrate.quad(lambda x: (-Q/(A*L))*phi_x_n(x,j), 0, L)[0] for j in range(N_x)]) for n in range(N_x)]

P0 = [sum([inv_r[n,j]*integrate.quad(lambda r: (ssr0(r)-(f0-1)*r/(a-R)-(R*f0-a)/(R-a))*phi_r_n(r,j), a, R)[0] for j in range(N_r)]) for n in range(N_r)]
P1 = [sum([inv_r[n,j]*integrate.quad(lambda r: (R-r)/(R-a)*phi_r_n(r,j), a, R)[0] for j in range(N_r)]) for n in range(N_r)]
P2 = [sum([inv_r[n,j]*integrate.quad(lambda r: -1/(a-R)*(kappa-Q/(pi*D))/r*phi_r_n(r,j), a, R)[0] for j in range(N_r)]) for n in range(N_r)]

tempix1 = np.array([I0[n] + f0*I1[n] for n in range(N_x)])
tempix2 = np.array([-I1[n] for n in range(N_x)])
tempix3 = np.array([labda_x[n]*I1[n]+I2[n] for n in range(N_x)])

tempir0 = np.array([P2[n]/labda_r[n] for n in range(N_r)])
tempir1 = np.array([P0[n]+f0*P1[n]-P2[n]/labda_r[n] for n in range(N_r)])
tempir2 = np.array([-P1[n] for n in range(N_r)])
tempir3 = np.array([labda_r[n]*P1[n] + P2[n] for n in range(N_r)])

PHIx = np.array([k*phi_x_dprime(0,n) + Q/A*phi_x_prime(0,n) for n in range(N_x)])
PHIr = np.array([kappa*phi_r_dprime(a,n) + (kappa-Q/(pi*D))/a*phi_r_prime(a,n) for n in range(N_r)])

Bx1 = np.array([tempix1[n]*PHIx[n] for n in range(N_x)])
Bx2 = np.array([tempix2[n]*PHIx[n] for n in range(N_x)])
Bx3 = np.array([tempix3[n]*PHIx[n] for n in range(N_x)])

Br0 = np.array([tempir0[n]*PHIr[n] for n in range(N_r)])
Br1 = np.array([tempir1[n]*PHIr[n] for n in range(N_r)])
Br2 = np.array([tempir2[n]*PHIr[n] for n in range(N_r)])
Br3 = np.array([tempir3[n]*PHIr[n] for n in range(N_r)])

<ipython-input-14-9a6c3751cb30>:13: RuntimeWarning: divide by zero encountered in double_scalars
  tempir0 = np.array([P2[n]/labda_r[n] for n in range(N_r)])
<ipython-input-14-9a6c3751cb30>:14: RuntimeWarning: divide by zero encountered in double_scalars
  tempir1 = np.array([P0[n]+f0*P1[n]-P2[n]/labda_r[n] for n in range(N_r)])


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [15]:
# c = 1/a*(kappa-Q/(pi*D))/(a-R)
# d = Q/(A*L)

# def trapx(ti, farr, tarr):
#     res = 0
#     if len(tarr)<3:
#         return 0
#     for n in range(N_x):
#         for i in range(1,len(tarr)-1):
#             res += farr[i]*np.exp(labda_x[n]*(tarr[i]-ti))
#         res = res*Bx3[n]
#     return dt*res

# def trapr(ti, farr, tarr):
#     res = 0
#     if len(tarr)<3:
#         return 0
#     for n in range(N_r):
#         for i in range(1,len(tarr)-1):
#             res += farr[i]*np.exp(labda_r[n]*(tarr[i]-ti))
#         res = res*Br3[n]
#     return dt*res

# def den(ti):
#     res = c - d
#     res += -np.sum(Bx2 + dt/2*Bx3)
#     res += np.sum(Br2 + dt/2*Br3)
#     return res

# def num(ti, farr, tarr):
#     res = c
#     res += np.sum(np.exp(-labda_x*ti)*Bx1 + f0*dt/2*np.exp(-labda_x*ti)*Bx3)
#     res += trapx(ti, farr, tarr)
#     res += -np.sum(Br0+np.exp(-labda_r*ti)*Br1 + f0*dt/2*np.exp(-labda_r*ti)*Br3)
#     res += -trapr(ti, farr, tarr)
#     return res

# farr = np.array([f0])
# tarr = np.array([0])
# dt = 1e-3
# M = 1000

# for i in range(M):
#     ti = tarr[-1]+dt
#     f_new = num(ti, farr, tarr)/den(ti)
#     farr = np.append(farr, np.array([f_new]))
#     tarr = np.append(tarr, np.array([ti]))

# plt.figure()
# plt.plot(tarr, farr)
# plt.hlines(ssx0(0), tarr[0], tarr[-1], 'k')
# plt.hlines(ssx(0), tarr[0], tarr[-1], 'k')
# plt.xlabel('t')
# plt.ylabel('f(t)')
# plt.show()

In [16]:
# from scipy.interpolate import CubicSpline

# cs = CubicSpline(tarr, farr)

# plt.figure()
# plt.plot(tarr, farr, '--')
# plt.plot(tarr, cs(tarr))
# plt.show()

# def g(t):
#     return cs(t)

# def g_prime(t):
#     dt = 1e-3
#     return (cs(t+dt)-cs(t))/dt

In [17]:
# x = np.linspace(0,L,20)
# r = np.linspace(a,R,10)

# ax = np.append(-x[::-1], r)
# res = np.append(sol_x(x,0)[::-1], sol_r(r,0))
# s0 = np.append(ssx0(x)[::-1], ssr0(r))
# sn = np.append(ssx(x)[::-1], ssr(r))

# plt.figure(figsize=(12,12))
# plt.plot(ax, res)
# plt.plot(ax, s0, '--', label='ss0')
# plt.plot(ax, sn, '--', label='ssn')
# for t in np.logspace(-1,3,8):
#     print(t)
#     xsol = sol_x(x,t)[::-1]
#     print('xsol done')
#     rsol = sol_r(r,t)
#     print('rsol done')
#     res = np.append(xsol, rsol)
#     plt.plot(ax, res, label=str(t))
#     plt.plot(0, g(t), 'r.')
# plt.xlim(-L,R)
# plt.ylim(0,1)
# plt.legend()
# plt.show()

In [18]:
# plt.figure()
# plt.plot([abs(PHIx[i]) for i in range(N_x)])
# plt.yscale('log')
# plt.show()

# plt.figure()
# plt.plot([abs(PHIr[i]) for i in range(N_r)])
# plt.yscale('log')
# plt.show()

In [19]:
#TEST
i1 = np.sum([inv_x[0,j] * integrate.quad(lambda x: (ssx0(x))*phi_x_n(x,j), 0, L)[0] for j in range(N_x)])
i2 = np.sum([inv_x[0,j] * integrate.quad(lambda x: (1-x/L)*phi_x_n(x,j), 0, L)[0] for j in range(N_x)])
i3 = np.sum([inv_x[0,j] * integrate.quad(lambda x: Q/(A*L)*phi_x_n(x,j), 0, L)[0] for j in range(N_x)])

p1 = np.sum([inv_r[0,j] * integrate.quad(lambda r: (ssr0(r))*phi_r_n(r,j), a, R)[0] for j in range(N_r)])
p2 = np.sum([inv_r[0,j] * integrate.quad(lambda r: (R-r)/(R-a)*phi_r_n(r,j), a, R)[0] for j in range(N_r)])
p3 = np.sum([inv_r[0,j] * integrate.quad(lambda r: 1/r*(kappa-Q/(pi*D))/(R-a)*phi_r_n(r,j), a, R)[0] for j in range(N_r)])

beta = -i2+p2
gamma = -i3+p3

print(gamma/beta)

-1.4533828021114656e-05


In [20]:
print(Q/(A*L))

7.5e-06
